# Hefei-NAMD Training Session 2020

![Hefei-NAMD LOGO](imgs/Hefei-NAMD.gif)

In this training session, we will learn how to do an NAMD calculation using Hefei-NAMD.

Outline of the session: 

* [A bit of theory](#a_bit_of_theory)

* [Installation of Hefei-NAMD](#installation)

* [An example: hot electron relaxation using Hefei-NAMD](#example)

* [Contact](#contact)

## A bit of theory <a id="a_bit_of_theory"></a>

### Flowchart of Hefei-NAMD

1. Build a supercell and perform the geometry optimization at 0 K.

2. Molecular dynamics equilibration run — canonical ensemble,

3. Molecular dynamics production run — microcanonical ensemble.

<img src="imgs/flowchart.png" width=300px />



## Installation<a id="installation"></a>

Hefei-NAMD is mainly written in Fortran and barely depends on any third-party library. The installation of Hefei-NAMD is very simple, all you need is a FORTRAN compiler. On your linux shell, do the following steps:

1. get the source code: 

   `git clone https://github.com/QijingZheng/Hefei-NAMD`
   

2. change to the source code directory

   `cd Hefei-NAMD/src/namd`
   
   Note that under the `"src"` directory, there are three sub-directories. For the moment, we will concentrate on the above one.


3. Just type `"make"` and you should have an executable named `"namd"` in the current directory. In case you want to access the executable from anywhere in the system, just add the path of the namd executable to the `"PATH"` environment variable, i.e. 

    `cat 'export PATH=/path/to/your/namd-executable:${PATH}' >> ~/.bashrc`

## An example <a id="example"></a>

## Contact<a id="contact"></a>

* Dr. Qijing Zheng (郑奇靖), <zqj@ustc.edu.cn>

* Mr. Xiang Jiang (蒋翔), <jxiang@mail.ustc.edu.cn>

* Dr. Weibin Chu (褚维斌)，<wbchu@ustc.edu.cn>

* Dr. Chuanyu Zhao (赵传寓)，<zhaochuanyu@zju.edu.cn>

* Prof. Jin Zhao (赵瑾), <zhaojin@ustc.edu.cn>

